In [1]:
debug = False

import ROOT as R
if debug:
    R.ROOT.EnableImplicitMT() # only for ROOT rdf

import os
from pprint import pprint
import pandas as pd 


Welcome to JupyROOT 6.19/01


In [2]:
# this cell is only for spark
import PyRDF
if not debug:
    PyRDF.use("spark")
PyRDF.include_headers("headers.hh")

print(PyRDF.current_backend.supported_operations)

['Define', 'Filter', 'Histo1D', 'Histo2D', 'Histo3D', 'Profile1D', 'Profile2D', 'Profile3D', 'Count', 'Sum', 'Fill', 'Snapshot', 'Graph', 'AsNumpy']


In [3]:
# this is only for root rdf "interactive"
if debug:
    R.gInterpreter.ProcessLine(".L headers.hh")

In [4]:
import latinos_rdf as lrdf

In [5]:
#samples = ["DATA","VBS", "VBF-V", "Wjets", "singleTop", "ttbar",  "VV","DY","FakeQCD", "VVV"]
samples = ["VBS", "Wjets", "DY", "singleTop", "ttbar"]
cut = "lowen_looseVBS"
output_basedir = "/eos/user/d/dmapelli/public/latino/Full2017_v1_200119/"

In [6]:
# # FIXME

columns_MC = ["events"]
#columns_MC = ["weight", "XSWeight", "TriggerEffWeight_1l", "btagSF"] + columns
#columns_DATA = columns
columns_MC = [ cut+"_"+c for c in columns_MC]

In [7]:
joblist = []

for sample in samples:
    if debug:
        trees, nfiles = lrdf.build_dataframe("Full2017v6", sample, R, "root") # ROOT RDF "interactive"
    else:
        trees, nfiles = lrdf.build_dataframe("Full2017v6",  sample, PyRDF , "pyrdf") # Spark
    for tree, nfile in zip(trees,nfiles):
        joblist.append((tree,nfile))
        
######## Now sort by number of files
jobslist = sorted(joblist, key=lambda v: v[1], reverse=True)


In [8]:
########
dfs = []
for tree,_ in jobslist:
        print(tree)
        # Choose the right output columns
        if sample == "DATA": outputcols = columns_DATA
        else: outputcols = columns_MC

        df = pd.DataFrame(tree[cut].rdf_node.AsNumpy(columns=outputcols))
        dfs.append(df)
        

Tree: Wjets
name: supercut
parent: None
cut: METFilter_MC && nLepton==1 && Lepton_pt[0]>30
vars: 
aliases: bVeto,bReq,bVetoSF,bReqSF,btagSF
weight: (puWeight*                   TriggerEffWeight_1l*                   Lepton_RecoSF[0]*                   EMTFbug_veto*Lepton_tightElectron_mvaFall17V1Iso_WP90_IdIsoSF[0]*                Lepton_tightMuon_cut_Tight_HWWW_IdIsoSF[0]*(Lepton_isTightElectron_mvaFall17V1Iso_WP90[0]>0.5 || Lepton_isTightMuon_cut_Tight_HWWW[0]>0.5)*PrefireWeight*btagSF)
--------------------------------------------------------------------------------
name: lowen_looseVBS
parent: supercut
cut: VBS_category==1             && vbs_0_pt > 30 && vbs_1_pt > 30              && vjet_0_pt > 30 && vjet_1_pt > 30             && mjj_vbs >=300                && deltaeta_vbs >= 2              && PuppiMET_pt > 30             && bVeto             && mjj_vjet > 65 && mjj_vjet < 105             
vars: lowen_looseVBS_events,lowen_looseVBS_Lepton_pt,lowen_looseVBS_Lepton_eta,lowen_looseVB

KeyboardInterrupt: 

Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


In [ ]:
all_df = dfs[0]
for df in dfs[1:]:
    all_df = all_df.append(df)  

all_df = all_df[ ( (all_df.lep_flavour == 11) & (all_df.lep_pt > 40))  | (all_df.lep_flavour == 13)]

all_df.to_pickle(output_basedir + sample + ".pkl")

In [9]:
dfs

[        lowen_looseVBS_events
 0                           1
 1                           1
 2                           1
 3                           1
 4                           1
 5                           1
 6                           1
 7                           1
 8                           1
 9                           1
 10                          1
 11                          1
 12                          1
 13                          1
 14                          1
 15                          1
 16                          1
 17                          1
 18                          1
 19                          1
 20                          1
 21                          1
 22                          1
 23                          1
 24                          1
 25                          1
 26                          1
 27                          1
 28                          1
 29                          1
 ...                       ...
 644900 